In [64]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-infobars")
# options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_experimental_option("prefs", { \
    "profile.default_content_setting_values.media_stream_mic": 2,     # 1:allow, 2:block 
    "profile.default_content_setting_values.media_stream_camera": 2,  # 1:allow, 2:block 
    "profile.default_content_setting_values.geolocation": 2,          # 1:allow, 2:block 
    "profile.default_content_setting_values.notifications": 2         # 1:allow, 2:block 
  })

import pandas as pd
import re

In [65]:
# declare global variables

WAIT_TIME = 5 # seconds

# default input & output file names
input_file = 'fender_bass_products.csv'
output_file = 'fender_bass_output.csv'

url = 'https://dealer.fender.com/login'
username = ''
password = ''

In [66]:
# create empty dataframe
df = pd.DataFrame(columns=['Product Name', 'Product Model', 'Product List Price', 'Product MAP Price', 'Product Cost', 'Product Description', 'Product Image URL'])

product_df = pd.read_csv(input_file)
product_df['Variant SKU'] = product_df['Variant SKU'].astype(str).apply(lambda x: re.sub( r'\.0$', '', x) )
product_list = product_df['Variant SKU'].tolist()

In [67]:
print(f'Scraping data for {len(product_list)} products:')
for product in product_list:
    print(f'\t{product}')

Scraping data for 42 products:
	190130800
	0190130809
	0190130872
	190142800
	0190142821
	0190142841
	193992763
	0193992718
	0193990705
	0193990700
	0193970700
	0193970705
	0193970755
	0193970706
	0193972761
	0193972705
	0193972718
	0193972700
	0193972763
	0193970719
	0190120802
	0190120857
	0190120800
	0198620380
	0198620357
	0198620345
	0198620300
	0198610380*
	0198612357*
	0198610300*
	0194000761
	0194000700
	0194000705
	0193932719
	0193932705
	0193932700
	0193932706
	0193930755
	0193930718
	0193930761
	0193930705
	0193930700


In [68]:
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
print(f'Opening {url} in Chrome browser...')
driver.get(url)
products_not_found = 0
products_not_found_list = []

C:\Users\AYNM\AppData\Local\Temp\ipykernel_17712\600678085.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
C:\Users\AYNM\AppData\Local\Temp\ipykernel_17712\600678085.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


Opening https://dealer.fender.com/login in Chrome browser...


In [69]:
# login
email_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id="emailInput"]')))
email_input.clear()
email_input.send_keys(username)

password_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id="passwordInput"]')))    
password_input.clear()
password_input.send_keys(password)

login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[id="submitLoginButton"]'))).click()

In [70]:
# search products
for product in product_list:
    if product == 'nan':
        continue
    search_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id="search-fld"]')))
    search_input.clear()
    search_input.send_keys(product)
    search_input.send_keys(Keys.RETURN)


    upc_span = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[name="imageView_upc"]')))
    upc = upc_span.text
    
    try:
        dealer_cost_span = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[name="imageView_dealerNet"]')))
        dealer_cost = dealer_cost_span.text.split('$')[1]
    except:
        dealer_cost = ''

    try:
        map_price_span = driver.find_element(By.CSS_SELECTOR, 'span[name="imageView_map"]')
        map_price = map_price_span.text.split('$')[1]
    except:
        map_price = ''
    
    model_no_span = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[name="imageView_modelNo"]')))

    model_no = model_no_span.text

    product_name = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'h3[name="imageView_name"]')))
    print(f'Product Name: {product_name.text}')
    print(f'Product Model: {model_no}')
    print(f'Dealer Cost: {dealer_cost}')
    print(f'Product Price: {map_price}')
    print(f'UPC: {upc}')

    img_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='product-image-container']/img[1]")))
    image_url = img_element.get_attribute('src')
    print(image_url)
    # dealer cost (span) (USD): name="imageView_dealerNet"
    # MAP price (span) (CAD): name="imageView_map"
    # model#/sku: (span) name="imageView_modelNo"
    # product name: (H3) name="imageView_name"
    # description: (div) class="description-column" .show_more_box.div.div.p
    # image url: (div) class="product-image-container" .img.src
    
    try:
        description_element = driver.find_element(By.XPATH, "//div[@class='description-column']/show-more-box/div/div[1]")
        description = description_element.text
    except:
        description = 'No description available'
    
    print(f'Product Description: {description}')
    df = df.append({'Product Name': product_name.text, 'Product Model': model_no, 'Product MAP Price': map_price, 'Product Cost': dealer_cost, 'UPC': upc, 'Product Description': description, 'Product Image URL': image_url}, ignore_index=True)

Product Name: AMERICAN ORIGINAL '60S JAZZ BASS®
Product Model: 0190130800
Dealer Cost: 
Product Price: 
UPC: 885978821068
https://www.fmicassets.com/Damroot/Med/10001/0190130800_gtr_frt_001_rr.png
Product Description: Released in 1960, the upscale Jazz Bass quickly found favor among working bassists thanks to its premium appointments and versatile sound. The narrower neck seemed tailor-made for fast, dexterous playing, while the pair of single-coil pickups added a bold new dimension to Fender bass tone. The American Original ‘60s Jazz Bass has all the tone and style of its earliest incarnation, with modern playability.


C:\Users\AYNM\AppData\Local\Temp\ipykernel_17712\909244397.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Product Name': product_name.text, 'Product Model': model_no, 'Product MAP Price': map_price, 'Product Cost': dealer_cost, 'UPC': upc, 'Product Description': description, 'Product Image URL': image_url}, ignore_index=True)


Product Name: AMERICAN ORIGINAL '60S JAZZ BASS®
Product Model: 0190130809
Dealer Cost: 
Product Price: 2,749.99
UPC: 885978841141
https://www.fmicassets.com/Damroot/Med/10001/0190130809_gtr_frt_001_rr.png
Product Description: Released in 1960, the upscale Jazz Bass quickly found favor among working bassists thanks to its premium appointments and versatile sound. The narrower neck seemed tailor-made for fast, dexterous playing, while the pair of single-coil pickups added a bold new dimension to Fender bass tone. The American Original ‘60s Jazz Bass has all the tone and style of its earliest incarnation, with modern playability.


C:\Users\AYNM\AppData\Local\Temp\ipykernel_17712\909244397.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Product Name': product_name.text, 'Product Model': model_no, 'Product MAP Price': map_price, 'Product Cost': dealer_cost, 'UPC': upc, 'Product Description': description, 'Product Image URL': image_url}, ignore_index=True)


Product Name: AMERICAN ORIGINAL '60S JAZZ BASS®
Product Model: 0190130872
Dealer Cost: 
Product Price: 2,749.99
UPC: 885978290932
https://www.fmicassets.com/Damroot/Med/10001/0190130872_gtr_frt_001_rr.png
Product Description: Released in 1960, the upscale Jazz Bass quickly found favor among working bassists thanks to its premium appointments and versatile sound. The narrower neck seemed tailor-made for fast, dexterous playing, while the pair of single-coil pickups added a bold new dimension to Fender bass tone. The American Original ‘60s Jazz Bass has all the tone and style of its earliest incarnation, with modern playability.


...

C:\Users\AYNM\AppData\Local\Temp\ipykernel_17712\909244397.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Product Name': product_name.text, 'Product Model': model_no, 'Product MAP Price': map_price, 'Product Cost': dealer_cost, 'UPC': upc, 'Product Description': description, 'Product Image URL': image_url}, ignore_index=True)


Product Name: AMERICAN PROFESSIONAL II PRECISION BASS®
Product Model: 0193930705
Dealer Cost: 
Product Price: 2,399.99
UPC: 885978579112
https://www.fmicassets.com/Damroot/Med/10001/0193930705_fen_ins_frt_1_rr.png
Product Description: The American Professional II Precision Bass® draws from more than sixty years of innovation, inspiration and evolution to meet the demands of today’s working player.
The ’63 P Bass profile neck now sports smooth rolled fingerboard edges, a “Super-Natural” satin finish and a newly sculpted neck heel for a supremely comfortable feel and easy access to the upper register. The new V-Mod II Precision Bass split-coil pickup is more articulate than ever while delivering the punch and growl the P Bass® is known for.
The American Pro II Precision Bass delivers instant familiarity and sonic versatility you’ll feel and hear right away, with broad ranging improvements that add up to nothing less than a new standard for professional instruments.


C:\Users\AYNM\AppData\Local\Temp\ipykernel_17712\909244397.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Product Name': product_name.text, 'Product Model': model_no, 'Product MAP Price': map_price, 'Product Cost': dealer_cost, 'UPC': upc, 'Product Description': description, 'Product Image URL': image_url}, ignore_index=True)


Product Name: AMERICAN PROFESSIONAL II PRECISION BASS®
Product Model: 0193930700
Dealer Cost: 
Product Price: 2,399.99
UPC: 885978436217
https://www.fmicassets.com/Damroot/Med/10001/0193930700_fen_ins_frt_1_rr.png
Product Description: The American Professional II Precision Bass® draws from more than sixty years of innovation, inspiration and evolution to meet the demands of today’s working player.
The ’63 P Bass profile neck now sports smooth rolled fingerboard edges, a “Super-Natural” satin finish and a newly sculpted neck heel for a supremely comfortable feel and easy access to the upper register. The new V-Mod II Precision Bass split-coil pickup is more articulate than ever while delivering the punch and growl the P Bass® is known for.
The American Pro II Precision Bass delivers instant familiarity and sonic versatility you’ll feel and hear right away, with broad ranging improvements that add up to nothing less than a new standard for professional instruments.


C:\Users\AYNM\AppData\Local\Temp\ipykernel_17712\909244397.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Product Name': product_name.text, 'Product Model': model_no, 'Product MAP Price': map_price, 'Product Cost': dealer_cost, 'UPC': upc, 'Product Description': description, 'Product Image URL': image_url}, ignore_index=True)


In [71]:
# save dataframe to CSV file
df.to_csv(output_file, index=False)
print('Results saved to CSV file\n')

print('Results:')
print(df)

print(f'Number of products found: {len(df)}\n')
print(f'Number of products not found: {products_not_found}\n')
if products_not_found > 0:
    print('Products not found:')
    for product in products_not_found_list:
        print(f'\t{product}')

# close browser
driver.close()
driver.quit()

Results saved to CSV file

Results:
                                    Product Name Product Model  \
0              AMERICAN ORIGINAL '60S JAZZ BASS®    0190130800   
1              AMERICAN ORIGINAL '60S JAZZ BASS®    0190130809   
2              AMERICAN ORIGINAL '60S JAZZ BASS®    0190130872   
3              AMERICAN ORIGINAL '70S JAZZ BASS®    0190142800   
4              AMERICAN ORIGINAL '70S JAZZ BASS®    0190142821   
5              AMERICAN ORIGINAL '70S JAZZ BASS®    0190142841   
6          AMERICAN PROFESSIONAL II JAZZ BASS® V    0193992763   
7          AMERICAN PROFESSIONAL II JAZZ BASS® V    0193992718   
8          AMERICAN PROFESSIONAL II JAZZ BASS® V    0193990705   
9          AMERICAN PROFESSIONAL II JAZZ BASS® V    0193990700   
10           AMERICAN PROFESSIONAL II JAZZ BASS®    0193970700   
11           AMERICAN PROFESSIONAL II JAZZ BASS®    0193970705   
12           AMERICAN PROFESSIONAL II JAZZ BASS®    0193970755   
13           AMERICAN PROFESSIONAL II JA